In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import json
import numpy as np
import pandas as pd
# Plot Images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
plt.style.use('ggplot')
import matplotlib
# Set GPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
# Plotly 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
cd ..

/home/anirudh/github/aip-eval


In [3]:
from studio.evaluation.keras import metrics, utils, visualizer
from studio.evaluation.keras.evaluators import CNNEvaluator
from studio.evaluation.keras.evaluators import Evaluator
from studio.evaluation.keras.evaluators import SequentialCNNEvaluator
from studio.evaluation.keras.evaluators import VisualQAEvaluator
from visual_qa.visual_qa import VisualQA

Using TensorFlow backend.


In [4]:
report_dir = 'consensus_evaluation_asl_v1_43_condtions/'
model_path = '/data/models/133_way_april_2019/ensemble/xception/model_max_acc_1gpu.h5'
visual_dictionary = '/home/anirudh/github/qa-data/data/visual_classifier/133_way_min_150_images_april_2019_model_map_diagnosis_nodes.json'
by_definition_csv = '/home/anirudh/github/qa-data/data/by-definition/by_def_unreviewed_43_conditions_asl.csv'
qa_data_json = '/home/anirudh/github/qa-data/data/testset/survey_testset_derm_22aug19_asl.json'
valid_evidence = '/home/anirudh/github/qa-data/data/by-definition/asl_valid_evidence.json'
visual_data_dir = '/data/datasets/visual_qa/all_cases_dup_removed_july_24_cropped'

In [5]:
cnn_evaluator = CNNEvaluator(
        concept_dictionary_path=None,
        custom_objects=None,
        concepts=None,
        model_path=model_path,
        batch_size=32,
        verbose=0)

/home/anirudh/github/aip-eval/.venv/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning:

No training configuration found in save file: the model was *not* compiled. Compile it manually.



In [6]:
visual_qa_evaluator = VisualQAEvaluator(
    report_dir=report_dir,
    qa_data_json = qa_data_json,
    visual_dictionary = visual_dictionary,
    by_definition_csv = by_definition_csv,
    valid_evidence = valid_evidence
)

In [7]:
with open('/home/anirudh/github/qa-data/data/visual_classifier/133_way_min_150_images_april_2019_model_map_diagnosis_nodes.json') as f:
    n_way = json.load(f)
id_class_name_dict = {}
for _class in n_way:
    id_class_name_dict[_class['diagnosis_id']] = _class['diagnosis_name']
filtered_diagnosis_ids = [_class['diagnosis_id'] for _class in n_way]

In [8]:
len(filtered_diagnosis_ids)

133

In [9]:
filtered_testset = [case for case in visual_qa_evaluator.filtered_qa_data if case['diagnosis_id'] in filtered_diagnosis_ids]

In [10]:
#qa_data_json = '/home/anirudh/github/qa-data/data/testset/survey_testset_derm_24july19.json'
len(filtered_testset)

2857

In [11]:
'''with open('/home/anirudh/github/qa-data/data/testset/custom_survey_testset_derm_24july19.json', 'w') as f:
    json.dump(filtered_testset, f, indent=4)
'''

"with open('/home/anirudh/github/qa-data/data/testset/custom_survey_testset_derm_24july19.json', 'w') as f:\n    json.dump(filtered_testset, f, indent=4)\n"

In [12]:
main_case = set()
for case in filtered_testset:
    main_case.add(case['case_id'].split('_')[0])

In [13]:
len(main_case)

1800

In [14]:
visual_diagnosis_ids = list(visual_qa_evaluator.visual_qa.visual_diagnosis_ids)
labels = []
image_paths = []
for sample in filtered_testset:
    image_paths.append(os.path.join(visual_data_dir, sample["image_id"]))
    labels.append(visual_diagnosis_ids.index(sample['diagnosis_id']))
    
probs = cnn_evaluator.predict(image_paths)
labels = np.array(labels)

Making predictions from model  0
88/89 [============================>.] - ETA: 2s
Processing remainder: 9


In [15]:
visual_result = cnn_evaluator.get_metrics(probs, labels, top_k=5, concept_labels=visual_qa_evaluator.visual_qa.visual_diagnosis_ids)

In [16]:
cnn_evaluator.plot_top_k_accuracy()

In [17]:
cnn_evaluator.show_results(mode='average')

,id,accuracy_top_1,accuracy_top_2,accuracy_top_3,accuracy_top_4,accuracy_top_5,weighted_precision,sensitivity,precision,f1_score,number_of_samples,number_of_classes
0,model_max_acc_1gpu.h5,0.359,0.481,0.556,0.606,0.652,0.414,0.269,0.298,0.352,2857,133


In [18]:
visual_qa_evaluator.filtered_qa_data = filtered_testset

In [19]:
combination_result = visual_qa_evaluator.evaluate(probs, differential=True)

In [20]:
utils.compare_visual_by_definition_results(visual_result['average']['accuracy'] ,combination_result['average']['accuracy'] )

,Mode,top_1,top_2,top_3,top_4,top_5
0,Visual,0.358768,0.480574,0.556178,0.605880,0.651733
1,Visual_QA,0.358768,0.483024,0.560378,0.613931,0.656283


In [21]:
combination_result

{'average': OrderedDict([('accuracy',
               [0.3587679, 0.4830242, 0.560378, 0.6139307, 0.6562828]),
              ('weighted_precision', 0.4137054),
              ('sensitivity', 0.2685478),
              ('precision', 0.3038564),
              ('f1_score', 0.3525533),
              ('number_of_samples', 2857),
              ('number_of_classes', 133),
              ('average_rank_change', -0.043402170108505424),
              ('Visual_Non_Zero_Probs', 379981),
              ('Visual_QA_Non_Zero_Probs', 340293),
              ('Absolute_Zero_change', -39688),
              ('Zero_change_percentage', -10.444732762953937)]),
 'individual': [{'concept': 'acne scarring',
   'metrics': OrderedDict([('sensitivity',
                 [0.7160494, 0.8395062, 0.9382716, 0.9506173, 0.9506173]),
                ('precision', 0.6516854),
                ('f1_score', 0.682353),
                ('TP', 58),
                ('FP', 31),
                ('FN', 23),
                ('n_samples', 